In [9]:
import geopandas as gpd
import os
import json
import pandas as pd
import numpy as np
import rasterio as rio
from rasterio.windows import Window
import uuid
from shapely.geometry import box
import matplotlib.pyplot as plt
from rasterio.plot import show
from rasterio.features import geometry_mask
from pycocotools import mask as coco_mask
from datetime import datetime
import base64

# Part 1: Proper labels
The labels in the Laurentian data are strings, but COCO works better if these are abstracted by integer classes. So we need to replace the strings with integers, and then save a map somewhere

In [2]:
def map_labels_to_integers(shapefile_paths, output_json_path):
    '''
    Runs an initial pass over all the shapefiles to get a list of classes, then
    maps them to integers for later use. Saves the map to a JSON file.

    Note: This also modifies the shapefiles by adding a 'Label' integer column.
    Back up the shapefiles before doing this, if it matters.
    '''
    label_to_int_map = {}
    int_to_label_map = {}
    next_int = 0  # Start mapping from integer 0
    
    for path in shapefile_paths:
        # Read the shapefile
        gdf = gpd.read_file(path)

        # Check if the 'Label' column exists
        if 'Label' not in gdf.columns:
            raise ValueError(f"The shapefile {path} does not contain a 'Label' column.")
        
        # Map the 'Label' to an integer if it's not already in the dictionary
        for label in gdf['Label']:
            if label not in label_to_int_map:
                label_to_int_map[label] = next_int
                int_to_label_map[next_int] = label
                next_int += 1
        
        # Replace the 'Label' column with the mapped integers
        gdf['class'] = gdf['Label'].map(label_to_int_map)

        # Save the modified shapefile
        gdf.to_file(path)

    # Save the label to integer mapping to a JSON file
    with open(output_json_path, 'w') as f:
        json.dump(int_to_label_map, f, indent=4)

    print(f"Label to integer mapping saved to {output_json_path}")


shapefile_paths = ['./data/Z1_polygons.gpkg', './data/Z2_polygons.gpkg', './data/Z3_polygons.gpkg']
output_json_path = './data/classes.json'
map_labels_to_integers(shapefile_paths, output_json_path)

Label to integer mapping saved to ./data/classes.json


# Part 2: Processing to COCO
There's a lot going on here, so I'll try to summarize it.

First, I have clipped the rasters a bit. There was some area around the edges that did not have labels, so they have been cut out. I only want to have trees in this dataset if they're labelled. This is why the geotiff paths end with `rgb-clipped.tif`.

We iterate through the (clipped geotiff, shapefile) pairs for each zone (Z1, Z2, Z3). For each geotiff, we need to process it in tiles. The tile width and height can be set at the top.

Each tile is given a UUID. This is necessary in case we want to add more datasets down the line, and avoid name collisions. We use `gpd.overlay` to calculate which shapefiles fall within this tile. We then do a bit of filtering to remove tiny bits of trees that are mostly not in the tile (this is the `intersecting_polygons = intersecting_polygons[intersecting_polygons['area'] >= 2.0]`).

The mask tool from `pycocotools` is used to do run-length encoding (RLE) on each polygon, after which we save it into an annotation entry. All info is appended to the coco JSON object.

In [12]:
def chunk_raster(geotiffs, shapefiles, name):
    if not os.path.exists(f'data/chunked/{name}'):
        os.mkdir(f'data/chunked/{name}')
    tile_width, tile_height = 1024, 1024

    # counters - we will need to modify these if we want to append more data to this dataset.
    tile_id = 0
    annotation_id = 0

    class_map_file = 'data/classes.json'

    with open(class_map_file, 'r') as f:
        class_map = json.load(f)

    coco_dict = {
        'info': [],
        'licenses': [],
        'images': [],
        'annotations': [],
        'categories': [{ 'id': idx, 'name': namex, 'supercategory': 'tree'} for idx, namex in class_map.items()],
    }

    for file, shapes in zip(geotiffs, shapefiles):
        
        gdf = gpd.read_file(shapes)

        # Create tiles from raster
        with rio.open(file) as tif:
            n_tiles_x = tif.width // tile_width
            n_tiles_y = tif.height // tile_height

            print(f'Parsing {n_tiles_y} x {n_tiles_x} tiles for {file}...')

            for j in range (n_tiles_y):
                print(f'  row {j}...')
                for i in range(n_tiles_x):
                    win_x = i * tile_width
                    win_y = j * tile_height
                    window = Window(win_x, win_y, tile_width, tile_height)
                    raster = tif.read(window=window)

                    # If it's >50% black pixels, just continue. No point segmenting it.
                    if np.sum(raster == 0) / (tile_width * tile_height * tif.count) > 0.5:
                        continue

                    window_transform = tif.window_transform(window)
                    tile_name = f'{uuid.uuid4()}.tif'

                    with rio.open(
                        f'data/chunked/{name}/{tile_name}', 'w',
                        driver='GTiff',
                        height=tile_height,
                        width=tile_width,
                        count=tif.count,
                        dtype=tif.dtypes[0],
                        crs=tif.crs,
                        transform=window_transform,
                    ) as tile_raster:
                        tile_raster.write(raster)
                        coco_dict['images'].append({
                            'id': tile_id,
                            'width': tile_width,
                            'height': tile_height,
                            'gsd': 0.0192, # resolution of the laurentian data
                            'file_name': tile_name,
                            'date_captured': '2022-06-01',
                        })

                        tile_annotations = []

                        # Get bbox for the whole tile, then use that to find relevant shape masks
                        bbox_polygon = gpd.GeoDataFrame(gpd.GeoSeries(box(*tile_raster.bounds)), columns=['geometry'], crs=tif.crs)
                        intersecting_polygons = gpd.overlay(gdf, bbox_polygon, how='intersection')
                        intersecting_polygons['area'] = intersecting_polygons.geometry.area

                        # We should drop tiny bits of masks sticking into the frame
                        intersecting_polygons = intersecting_polygons[intersecting_polygons['area'] >= 2.0]
                        intersecting_polygons.reset_index(inplace=True, drop=True)

                        for i, shp in intersecting_polygons.iterrows():
                            mask = geometry_mask([shp.geometry], transform=window_transform, invert=True, out_shape=(tile_height, tile_width)).astype(np.uint8)
                            segmentation = coco_mask.encode(np.asfortranarray(mask))
                            segmentation['counts'] = segmentation['counts'].decode('ascii')
                            #segmentation['counts'] = base64.b64encode(segmentation['counts']).decode('utf-8')

                            # Get bounding box - some dubious operations here
                            minx_lat, miny_lat, maxx_lat, maxy_lat = shp.geometry.bounds
                            miny, minx = rio.transform.rowcol(window_transform, minx_lat, miny_lat)
                            maxy, maxx = rio.transform.rowcol(window_transform, maxx_lat, maxy_lat)

                            # Annotation entry for this shape
                            tile_annotations.append({
                                'id': annotation_id,
                                'image_id': tile_id,
                                'category_id': shp['class'],
                                'segmentation': segmentation,
                                'area': shp['area'],
                                'bbox': [minx, miny, abs(maxx-minx), abs(maxy-miny)],
                                'iscrowd': 0,
                            })
                            annotation_id += 1
                        
                        tile_id += 1
                        coco_dict['annotations'].extend(tile_annotations)

                        '''
                        # Some graphing code in case I need it again
                        fig, ax = plt.subplots(figsize=(10,10))
                        show(raster, ax=ax, transform=window_transform)
                        show(mask, ax=ax, transform=window_transform, alpha=0.5)
                        #intersecting_polygons.loc[[8]].plot(ax=ax, edgecolor='black', alpha=0.5)
                        '''
            
            # Write JSON after every file, for funsies
            with open(f'data/chunked/{name}.json', 'w') as f:
                json.dump(coco_dict, f)



In [13]:
train_geotiff_paths = ['./data/2021-06-17/zone1/2021-06-17-sbl-z1-rgb-clipped.tif', './data/2021-06-17/zone2/2021-06-17-sbl-z2-rgb-clipped.tif']
train_shapefile_paths = ['./data/Z1_polygons.gpkg', './data/Z2_polygons.gpkg']

val_geotiff_paths = ['./data/2021-06-17/zone3/2021-06-17-sbl-z3-rgb-clipped.tif']
val_shapefile_paths = ['./data/Z3_polygons.gpkg']

chunk_raster(train_geotiff_paths, train_shapefile_paths, 'train')
chunk_raster(val_geotiff_paths, val_shapefile_paths, 'val')

Parsing 30 x 29 tiles for ./data/2021-06-17/zone1/2021-06-17-sbl-z1-rgb-clipped.tif...
  row 0...
  row 1...
  row 2...
  row 3...
  row 4...
  row 5...
  row 6...
  row 7...
  row 8...
  row 9...
  row 10...
  row 11...
  row 12...
  row 13...
  row 14...
  row 15...
  row 16...
  row 17...
  row 18...
  row 19...
  row 20...
  row 21...
  row 22...
  row 23...
  row 24...
  row 25...
  row 26...
  row 27...
  row 28...
  row 29...
Parsing 19 x 29 tiles for ./data/2021-06-17/zone2/2021-06-17-sbl-z2-rgb-clipped.tif...
  row 0...
  row 1...
  row 2...
  row 3...
  row 4...
  row 5...
  row 6...
  row 7...
  row 8...
  row 9...
  row 10...
  row 11...
  row 12...
  row 13...
  row 14...
  row 15...
  row 16...
  row 17...
  row 18...
Parsing 18 x 17 tiles for ./data/2021-06-17/zone3/2021-06-17-sbl-z3-rgb-clipped.tif...
  row 0...
  row 1...
  row 2...
  row 3...
  row 4...
  row 5...
  row 6...
  row 7...
  row 8...
  row 9...
  row 10...
  row 11...
  row 12...
  row 13...
  row 14...
 